# Highcode Example: Save Plots as HTML or PNG

In [ ]:
## ----------------------------------------------------------------
## Install or update packages(recommended to run in Terminal)
## ----------------------------------------------------------------
!pip show modeva
# !pip uninstall modeva
#!pip install modeva

In [ ]:
## ----------------------------------------------------------------
## Data Loading: e.g. built-in BikeSharing dataset
## ----------------------------------------------------------------
from modeva import DataSet
ds = DataSet()
ds.load(name="BikeSharing")

In [ ]:
## ----------------------------------------------------------------
## EDA (Exploratory Data Analysis)
## ----------------------------------------------------------------

# Example EDA plot: univariate histogram
result = ds.eda_1d(feature="cnt", plot_type="histogram")

# Display chart on screen
result.plot(figsize=(6,4))

# Save plot as HTML or PNG
result.plot_save(file_name="BS_images/cnt_histogram", format='html', figsize=(6, 4))
result.plot_save(file_name="BS_images/cnt_histogram", format='png', figsize=(6, 4))

In [ ]:
## ----------------------------------------------------------------
## Data Processing: refer to Highcode-BikeSharing.ipynb
## ----------------------------------------------------------------
ds.scale_numerical(method="minmax")
ds.encode_categorical(features=("season", "weathersit", "holiday", "workingday"), method="ordinal")
ds.bin_numerical(features=("atemp", ), bins=10, method="uniform")
ds.preprocess()
ds.set_inactive_features(features=['season', 'workingday', 'temp'])
ds.set_target(feature="cnt")
ds.set_task_type('Regression')
ds.set_random_split(test_ratio=0.33)

In [ ]:
## ----------------------------------------------------------------
## LGBM-Depth2 and Interpretability 
## ----------------------------------------------------------------
from modeva.models import MoLGBMRegressor
model_lgbm = MoLGBMRegressor(name="LGBM", max_depth=2, n_estimators=100, verbose=-1)
model_lgbm.fit(ds.train_x, ds.train_y.ravel())

from modeva import FactSheet
fs = FactSheet(ds, model_lgbm)

result1 = fs.interpret_fi()
result1.plot(figsize=(6,4))
result1.plot_save(file_name="BS_images/LGBM_feature_importance", format='html', figsize=(6, 4))
result1.plot_save(file_name="BS_images/LGBM_feature_importance", format='png', figsize=(6, 4))

result2 = fs.interpret_effects(features="hr")
result2.plot(figsize=(6, 4))
result2.plot_save(file_name="BS_images/LGBM_effect_plot_hr", format='html', figsize=(6, 4))
result2.plot_save(file_name="BS_images/LGBM_effect_plot_hr", format='png', figsize=(6, 4))

In [ ]:
## ----------------------------------------------------------------
## MoE Benchmarking
## ----------------------------------------------------------------
from modeva.models import MoMoERegressor
model_moe = MoMoERegressor(name="MOE", max_depth=2, n_clusters=5, n_estimators=100, verbose=-1)
model_moe.fit(ds.train_x, ds.train_y.ravel())

In [ ]:
# Example of generating/saving multiple plots 

from modeva import FactSheet
fsc = FactSheet(ds, models=[model_lgbm, model_moe])

result = fsc.compare_accuracy_table(train_dataset="train", test_dataset="test", metric=("MAE", "MSE", "R2"))

result.plot(figsize=(6,4))
result.plot_save(file_name="BS_images/MoE_Benchmark_Accuracy", format='html', figsize=(6, 4))
result.plot_save(file_name="BS_images/MoE_Benchmark_Accuracy", format='png', figsize=(6, 4))